Processes and sanity checks for run 03, the run for which a better $w$ is found. Code here is largely based on `./output_investigation_02.ipynb` and `./feasible_c.ipynb`.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
from scipy import stats
from scipy.optimize import minimize

In [2]:
from generate_opt_objects import A_b_generation

**PNC:/Users/mikestanley/anaconda3/envs/mc_sampling/lib/python3.8/site-packages/PseudoNetCDF/pncwarn.py:24:UserWarning:
  pyproj could not be found, so IO/API coordinates cannot be converted to lat/lon; to fix, install pyproj or basemap (e.g., `pip install pyproj)`


In [3]:
# results base directory
BASE_DIR = './data/carbon_flux/results'

In [5]:
# paths
OBS_FP = '../fixed_optimization_inputs/y_affine_corrected.npy'

In [6]:
# get necessary objects
with open(OBS_FP, 'rb') as f:
    y_obs = np.load(f)

CONSTR_DIR = '/Users/mikestanley/Research/Carbon_Flux/optimization/data/sign_corrected'
A, b = A_b_generation(
    box_constraint_fp=CONSTR_DIR + '/scipy_bnds.pkl'
)

FUNC_FP = '/Users/mikestanley/Research/Carbon_Flux/optimization'
FUNC_FP += '/src_admm/data/carbon_flux/objects/na_june_functional.npy'
with open(FUNC_FP, 'rb') as f:
    h = np.load(f)

In [17]:
# psi_alpha
with open('./data/carbon_flux/objects/opt_res_cont.pkl', 'rb') as f:
    opt_slack = pickle.load(f)
    
PSI_ALPHA = np.sqrt(stats.chi2.ppf(q=.95, df=1) + opt_slack[1])

In [19]:
def f_w(w, y=y_obs, psi_alpha=PSI_ALPHA, b=b, c=np.zeros_like(b)):
    """ Lower endpoint functional """
    return np.dot(w, y) - psi_alpha * np.linalg.norm(w) - np.dot(b, c)

# Computing most feasible $c$ given starting $w$

In [20]:
# read in the starting w
with open('./data/carbon_flux/objects/w_vec_max.npy', 'rb') as f:
    w_start = np.load(f)

In [8]:
# read in computed KTw
with open('./data/carbon_flux/objects/KTw_for_w_max.npy', 'rb') as f:
    KTw = np.load(f)
    
print(KTw.shape)

(26496,)


In [11]:
def obj_func(c, h=h, A=A, KTw=KTw):
    return np.dot(h + A.T @ c - KTw, h + A.T @ c - KTw)

def obj_jac(c, h=h, A=A, KTw=KTw):
    return 2 * A @ (h + A.T @ c - KTw)

In [16]:
# perform the optimization
c_opt_res_0_nn = minimize(
    fun=obj_func, x0=np.zeros_like(b), method='L-BFGS-B',
    jac=obj_jac, bounds=[(0, None)] * 11120,
    options={'iprint': 1}
)

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =        11120     M =           10

At X0     11120 variables are exactly at the bounds

At iterate    0    f=  4.36046D+06    |proj g|=  4.06870D+02

At iterate    1    f=  3.05261D+06    |proj g|=  3.40428D+02

At iterate    2    f=  6.84313D+00    |proj g|=  6.82121D-13

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
11120      2      7      2     0   139   6.821D-13   6.843D+00
  F =   6.8431277102822898     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            


In [21]:
# what is the new objective function value with this c?
f_w(w=w_start, c=c_opt_res_0_nn['x'])

-39571756.7549447